# Logistic Regression with Newton's Method

Objective: create logistic regression model from scratch using Newton's method to optimize weights to minimize cross-entropy loss.

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score

First we will create functions to compute the log likelihood, gradient, and hessian, as well as the logistic regression train and predict functions.

In [2]:
def loglikelihood(p, y):
    l = np.sum(y.dot(np.log(p)) + (1-y).dot(np.log(1-p)))
    return l

In [3]:
def gradient(p, x, y):
    y = np.expand_dims(y, axis=-1)
    grad = np.array([[np.sum((y - p) * x), np.sum(y - p)]])
    return grad

In [4]:
def hessian(p, x, y):
    H = np.array([[np.sum(p * (1 - p) * x * x), np.sum(p * (1 - p) * x)], [np.sum(p * (1 - p) * x), np.sum(p * (1 - p))]])
    return H

In [5]:
def logistic_reg_train(X, y):
    
    # initialize weights    
    w0 = 0
    w1 = np.ones((X.shape[1], 1))

    # calculate sigmoid probability and log likelihood
    a = X.dot(w1) + w0
    p = 1 / (1 + np.exp((-a).astype(float)))
    l = loglikelihood(p, y)
    
    delta = 100
    while delta > 0.000001:
        # calculate gradient and hessian
        g = gradient(p, X, y)
        H = hessian(p, X, y)
        
        # compute our step
        w_new = (np.linalg.inv(H)).dot(g.T)
        
        # update weights
        w0 += w_new[1]
        w1 += w_new[0]

        # compute new sigmoid probability and log likelihood with new weights
        a_new = X.dot(w1) + w0
        p_new = 1 / (1 + np.exp((-a_new).astype(float)))
        l_new = loglikelihood(p, y)
        delta = abs(l - l_new)
        l = l_new
        
    return (w0, w1)

In [6]:
def logistic_reg_predict(w0, w1, X, y):
    
    # compute sigmoid probability
    a = X_test.dot(w1)  + w0
    sig = 1 / (1 + np.exp((-a).astype(float)))
    
    # record labels as 0 if the probability is less than 0.5 and 1 otherwise
    y_pred = np.array([0 if s < 0.5 else 1 for s in sig])
    
    # calculate accuracy between computed labels and ground truth labels
    correct = 0
    for i in range(len(y)):
        if y_pred[i] == y[i]:
            correct += 1
            
    acc = correct / len(y)
    
    return acc

Next we will prepare and clean the dataset. Column 7 of the dataset contains 16 tuples with a missing value, which has been replaced with the character '?'. Because of this, each entry of column 7 is stored as a string instead of an integer. To remedy this, we will calculate the mean of that column and replace each missing value with the mean, as well as convert each existing value to an integer.

In [7]:
df = pd.read_csv('breast-cancer-wisconsin.data', header = None)
data = np.array(df)

# First calculate the mean of the column
col = data[:,6]
cnt = 0
idx = 0
missing = 0
while idx < (len(col)-1):
    if col[idx] == '?':
        idx += 1
        missing += 1
    else:
        cnt += int(col[idx])
        idx += 1
mean = cnt // (len(col) - missing)
        
# If the value is missing, replace it with the mean, otherwise convert the value to an integer
for x in range(col.shape[0]):
    if data[x][6] == '?':
        data[x][6] = mean
    else:
        data[x][6] = int(data[x][6])

The class labels in this dataset are stored as a 2 for benign and 4 for malignant, so now we will split the data between X (data) and y (labels) and convert those labels to 0 and 1.

In [8]:
# take columns 1-9 as our data X, and our last column as the labels y.
X = data[:, 1:-1]
y = data[:, 10]

# convert labels from 2 and 4 to 0 and 1
for i in range(len(y)):
    if y[i] == 2:
        y[i] = 0
    else:
        y[i] = 1

print(X.shape, y.shape)

(699, 9) (699,)


Now we are ready to train and predict with the logistic regression model. We want to train and predict ten times, each time with a new random split of data between the training and test sets.

In [9]:
# array to store accuracies
accs = []

for i in range(10):
    
    # shuffle the data
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    
    # create 80-20 split between training set and test set 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    
    # train, predict, and store the accuracy
    w0, w1 = logistic_reg_train(X_train, y_train)
    acc = logistic_reg_predict(w0, w1, X_test, y_test)
    accs.append(acc)
    
print('Accuracies: ' , accs)
print('Average Accuracy:', np.mean(accs))

Accuracies:  [0.9785714285714285, 0.9642857142857143, 0.9785714285714285, 0.9785714285714285, 0.9785714285714285, 0.9785714285714285, 0.95, 0.9714285714285714, 0.9571428571428572, 0.9714285714285714]
Average Accuracy: 0.9707142857142858


C:\Users\carme\AppData\Local\Temp\ipykernel_24924\25609289.py:2: RuntimeWarning: divide by zero encountered in log
  l = np.sum(y.dot(np.log(p)) + (1-y).dot(np.log(1-p)))
C:\Users\carme\AppData\Local\Temp\ipykernel_24924\1009655005.py:27: RuntimeWarning: overflow encountered in exp
  p_new = 1 / (1 + np.exp((-a_new).astype(float)))
C:\Users\carme\AppData\Local\Temp\ipykernel_24924\3241350734.py:5: RuntimeWarning: overflow encountered in exp
  sig = 1 / (1 + np.exp((-a).astype(float)))


We can use the SciKit-Learn Logistic Regression model and compare our accuracies.

In [10]:
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
model = LogisticRegression().fit(X_train.astype('int'), y_train.astype('int'))
scores = model.predict(X_test.astype('int'))

print('Accuracy: ' , accuracy_score(y_test.astype('int'), scores.astype('int')))

Accuracy:  0.9714285714285714


Let's try this same process on normalized data.

In [11]:
# array to store accuracies
accs = []

for i in range(10):
    
    # shuffle the data
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    
    # create 80-20 split between training set and test set 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    X_train = preprocessing.normalize(X_train, axis=0)
    X_test = preprocessing.normalize(X_test, axis=0)
    
    # train, predict, and store the accuracy
    w0, w1 = logistic_reg_train(X_train, y_train)
    acc = logistic_reg_predict(w0, w1, X_test, y_test)
    accs.append(acc)
    
print('Accuracies: ' , accs)
print('Average Accuracy:', np.mean(accs))

Accuracies:  [0.6, 0.6357142857142857, 0.6571428571428571, 0.6714285714285714, 0.6428571428571429, 0.6714285714285714, 0.6071428571428571, 0.7428571428571429, 0.6428571428571429, 0.6714285714285714]
Average Accuracy: 0.6542857142857144


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train = preprocessing.normalize(X_train, axis=0)
X_test = preprocessing.normalize(X_test, axis=0)
model = LogisticRegression().fit(X_train.astype('int'), y_train.astype('int'))
scores = model.predict(X_test.astype('int'))

print('Accuracy: ' , accuracy_score(y_test.astype('int'), scores.astype('int')))

Accuracy:  0.6142857142857143


With the normalized data we do not get a Runtime Warning, but we also get a much lower accuracy.